In [ ]:
import wrds

In [ ]:
db = wrds.Connection()
db.list_libraries() 
db.list_tables(library='frb')

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


['fx_daily', 'fx_monthly', 'rates_daily', 'rates_monthly']

In [ ]:
db.describe_table(library='frb',table='fx_daily').head(5)

Approximately 14115 rows in frb.fx_daily.


,name,nullable,type,comment
0,date,True,DATE,date
1,dexusal,True,DOUBLE PRECISION,Exchange Rate for Australia (US$/Australian$)
2,dexalus,True,DOUBLE PRECISION,Exchange Rate for Australia (Australian$/US$)
3,dexbzus,True,DOUBLE PRECISION,Exchange Rate for Brazil (Real/US$)
4,dexcaus,True,DOUBLE PRECISION,Exchange Rate for Canada (C$/US$)
5,dexchus,True,DOUBLE PRECISION,Exchange Rate for People's Republic of China (...
6,dexdnus,True,DOUBLE PRECISION,Exchange Rate for Denmark (Krone/US$)
7,dexhkus,True,DOUBLE PRECISION,Exchange Rate for Hong Kong (Dollar/US$)
8,dexinus,True,DOUBLE PRECISION,Exchange Rate for India (Rupee/US$)
9,dexjpus,True,DOUBLE PRECISION,Exchange Rate for Japan (Yen/US$)


In [ ]:
columns = db.get_table(library='frb', table='fx_daily').columns
print(columns)

Index(['date', 'dexusal', 'dexalus', 'dexbzus', 'dexcaus', 'dexchus',
       'dexdnus', 'dexhkus', 'dexinus', 'dexjpus', 'dexkous', 'dexmaus',
       'dexmxus', 'dexusnz', 'dexnzus', 'dexnous', 'dexsius', 'dexsfus',
       'dexslus', 'dexsdus', 'dexszus', 'dextaus', 'dexthus', 'dexusuk',
       'dexukus', 'dexvzus', 'exauus', 'exbeus', 'exfnus', 'exfrus', 'exgeus',
       'exgrus', 'exusir', 'exirus', 'exitus', 'exneus', 'expous', 'exspus',
       'exusec', 'execus', 'dexuseu', 'dexeuus', 'dtwexb', 'dtwexbgs',
       'dtwexm', 'dtwexafegs', 'dtwexo', 'dtwexemegs', 'indexgx'],
      dtype='object')


In [ ]:
fx = db.raw_sql("""
    SELECT date, dexuseu, dexusuk, dexjpus
    FROM frb.fx_daily
    WHERE date >= '2000-01-01'
    ORDER BY date
""")

# Rename columns for clarity
fx = fx.rename(columns={
    'dexuseu': 'EURUSD',   # USD per Euro
    'dexusuk': 'GBPUSD',   # USD per British Pound
    'dexjpus': 'USDJPY'    # Yen per USD
})


In [ ]:
fx.head(5)

,date,EURUSD,GBPUSD,USDJPY
0,2000-01-03,1.0155,1.627,101.7
1,2000-01-04,1.0309,1.637,103.09
2,2000-01-05,1.0335,1.6415,103.77
3,2000-01-06,1.0324,1.6475,105.19
4,2000-01-07,1.0294,1.6384,105.17
